In [25]:
import numpy as np
from torch import nn
from torch.nn import MSELoss
%matplotlib notebook
from func import *
import torch
from torch.utils.data import TensorDataset, DataLoader

In [26]:
np.random.seed(42)
torch.manual_seed(42)

In [27]:
def loss_func(speed, theta, t_drop, t_detonate):
    set_fy_mission(0, speed, theta)
    interval = get_time_interval(0, 0, t_drop, t_detonate)
    period = get_lebesgue(interval)
    loss = -period
    return loss

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.ModuleList([
            nn.Linear(4, 64),
            nn.Linear(64, 256),
            nn.Linear(256, 512),
            nn.Linear(512, 32),
            nn.Linear(32, 4),
            nn.Linear(4, 1),
        ])
        self.relu = nn.ReLU()

    def forward(self, x):
        for layer in self.linear[:-1]:
            x = self.relu(layer(x))
        x = self.linear[-1](x)
        return x

In [29]:
"""para_min = np.array([fy_speed_min, 0, 0, 0])
para_max = np.array([fy_speed_max, 2*np.pi, t_max, t_max])
num_dataset = 1000000
x_set = para_min + (para_max - para_min) * np.random.rand(num_dataset, 1)
y_set = np.zeros([num_dataset])
for i in range(num_dataset):
    print('\r', i, end='')
    y_set[i] = loss_func(*x_set[i])
    if i % 100 == 0:
        np.savez(f"nn_data{i % 1000}.npz", x=x_set, y=y_set)"""

'para_min = np.array([fy_speed_min, 0, 0, 0])\npara_max = np.array([fy_speed_max, 2*np.pi, t_max, t_max])\nnum_dataset = 1000000\nx_set = para_min + (para_max - para_min) * np.random.rand(num_dataset, 1)\ny_set = np.zeros([num_dataset])\nfor i in range(num_dataset):\n    print(\'\r\', i, end=\'\')\n    y_set[i] = loss_func(*x_set[i])\n    if i % 100 == 0:\n        np.savez(f"nn_data{i % 1000}.npz", x=x_set, y=y_set)'

In [30]:
data = np.load("nn_data0.npz")
X_tensor = torch.from_numpy(data["x"]).float()  # 转换为float32类型
y_tensor = torch.from_numpy(data["y"]).float()   # 转换为long类型（用于分类）
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [31]:
net = Net()

In [ ]:
net.train()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
loss_fc = MSELoss()
for epoch in range(100):
    print('\r', epoch, end=' ')
    epoch_loss = 0

    for x, y in dataloader:

        net_y = net(x)
        loss = loss_fc(net_y, y)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(epoch_loss / len(dataloader))

 0 0.2128387729998237
 1 0.2127321745548993
 2 0.21318830012841258
 3 0.21271856794300587
 4 0.21272044043803315
 5 0.21331413939738197
 6 0.2127427955774634
 7 0.21280489375403744
 8 0.21271786551380448
 9 0.21271989194116295
 10 0.2130564153263929
 11 0.21302315323148915
 12 0.21271485460898293
 13 0.21272075212330246
 14 0.21302861461876585
 15 0.21271744968820372
 16 0.21272008363524877
 17 0.21271989217995088
 18 0.2129409067597844
 19 0.21271998890445257
 20 

In [105]:
net.eval()
with torch.no_grad():
    res = 0
    for x, y in dataloader:
        net_y = net(x)
        res += torch.abs(net_y - y).sum()
    print(res / len(dataloader))

tensor(127.0951)
